In [2]:
!pip install segmentation_models

     |████████████████████████████████| 51kB 2.3MB/s 


In [16]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import re
import collections
import io
import pandas as pd
import numpy as np
from keras.applications.resnet50 import ResNet50,preprocess_input,decode_predictions
from keras.models import Model,load_model
from time import time
from keras.preprocessing import image
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
from keras.layers.merge import add
from keras.utils import generic_utils
import segmentation_models as sm
import matplotlib.pyplot as plt
from google.colab import files
import pickle

In [12]:
files.upload()

Saving model_19.h5 to model_19.h5


In [14]:
files.upload()

Saving idx_to_word.pkl to idx_to_word.pkl
Saving word_to_idx.pkl to word_to_idx.pkl


{'idx_to_word.pkl': b'\x80\x03}q\x00(K\x01X\x02\x00\x00\x00inq\x01K\x02X\x03\x00\x00\x00theq\x02K\x03X\x02\x00\x00\x00onq\x03K\x04X\x02\x00\x00\x00isq\x04K\x05X\x03\x00\x00\x00andq\x05K\x06X\x03\x00\x00\x00dogq\x06K\x07X\x04\x00\x00\x00withq\x07K\x08X\x03\x00\x00\x00manq\x08K\tX\x02\x00\x00\x00ofq\tK\nX\x03\x00\x00\x00twoq\nK\x0bX\x05\x00\x00\x00whiteq\x0bK\x0cX\x05\x00\x00\x00blackq\x0cK\rX\x03\x00\x00\x00boyq\rK\x0eX\x03\x00\x00\x00areq\x0eK\x0fX\x05\x00\x00\x00womanq\x0fK\x10X\x04\x00\x00\x00girlq\x10K\x11X\x02\x00\x00\x00toq\x11K\x12X\x07\x00\x00\x00wearingq\x12K\x13X\x02\x00\x00\x00atq\x13K\x14X\x06\x00\x00\x00peopleq\x14K\x15X\x05\x00\x00\x00waterq\x15K\x16X\x03\x00\x00\x00redq\x16K\x17X\x05\x00\x00\x00youngq\x17K\x18X\x05\x00\x00\x00brownq\x18K\x19X\x02\x00\x00\x00anq\x19K\x1aX\x03\x00\x00\x00hisq\x1aK\x1bX\x04\x00\x00\x00blueq\x1bK\x1cX\x04\x00\x00\x00dogsq\x1cK\x1dX\x07\x00\x00\x00runningq\x1dK\x1eX\x07\x00\x00\x00throughq\x1eK\x1fX\x07\x00\x00\x00playingq\x1fK X\x05\x00\x00\x

In [28]:
with open('/content/idx_to_word.pkl','rb') as i2w:
  idx_to_word = pickle.load(i2w)

In [26]:
with open('/content/word_to_idx.pkl','rb') as w2i:
  word_to_idx = pickle.load(w2i)

In [19]:
model = load_model('/content/model_19.h5')

In [21]:
files.upload()

Saving temp12.jpg to temp12.jpg


{'temp12.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x13\x12\x15\x13\x13\x12\x15\x15\x15\x17\x18\x17\x18\x17\x18\x18\x18\x16\x18\x18\x18\x17\x17\x15\x18\x17\x15\x1a\x18\x18\x18\x1d( \x18\x1a%\x1d\x17\x15!1!%)+...\x17\x1f383-7(-.+\x01\n\n\n\x0e\r\x0e\x1a\x10\x10\x1b-% %--------/-----------------------------------------\xff\xc0\x00\x11\x08\x00\xae\x01"\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x02\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x01\x02\x05\x06\x00\x07\x08\xff\xc4\x00=\x10\x00\x01\x03\x02\x03\x05\x05\x06\x04\x04\x06\x03\x00\x00\x00\x00\x01\x00\x02\x11\x03!\x04\x121\x05AQaq\x13"\x81\x91\xa1\x062\xb1\xc1\xd1\xf0BRb\xe1\x14\x15C\x82\x07$3r\xa2\xf1#\x92\xb2\xff\xc4\x00\x1a\x01\x00\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\xff\xc4\x00&\x11\x00\x02\x02\x02\x02\x02\x03\x00\x02\x03\x01\x00\x00\x00\x00\x00\x00\x01\x02\x11\x03\x12\x13!\

In [20]:
!ls

idx_to_word.pkl  model_19.h5  sample_data  word_to_idx.pkl


In [4]:
model_temp = ResNet50(weights='imagenet',input_shape=(224,224,3))
model_temp.summary()

102973440/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [5]:
model_resnet = Model(model_temp.input,model_temp.layers[-2].output)

In [6]:
def preprocess_img(img):
  img = image.load_img(img,target_size=(224,224)) ## load image
  img = image.img_to_array(img)
  img = np.expand_dims(img,axis=0) ## it converts image in shape of a batch

  ## normalization
  img = preprocess_input(img)
  return img


In [9]:
def encode_img(img):
  img = preprocess_img(img)
  pred_vec = model_resnet.predict(img)
  pred_vec = pred_vec.reshape((1,pred_vec.shape[1])) 
  return pred_vec 

In [24]:
def pred(photo):
  in_text = 'startseq'
  max_len=35

  for i in range(max_len):
    sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
    sequence = pad_sequences([sequence],maxlen=max_len,padding='post')

    ypred = model.predict([photo,sequence])
    ypred = ypred.argmax()
    word = idx_to_word[ypred]
    in_text += (' '+word)

    if word == 'endseq':
      break
  final_cap = in_text.split()[1:-1]
  final_cap = ' '.join(final_cap)
  return final_cap

In [22]:
 enc = encode_img('/content/temp12.jpg')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [31]:
pred(enc)


'man is climbing up rocky hill'